# EDA — Issue 1: inspección general y calidad básica

En este notebook se cubre el **Issue 1** del bloque de EDA:

- Cargar dataset (si no está ya cargado en memoria)
- Revisar dimensiones
- Identificar tipos de variables
- Visualización general de datos
- Conteo de nulos por variable
- Conteo de valores únicos por variable
- Identificación de columnas constantes o poco informativas

> Alcance: análisis descriptivo inicial, sin transformaciones relevantes para modelado.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

sns.set(style='whitegrid', context='notebook')

## 1) Carga del dataset

In [ ]:
from pathlib import Path

path_candidates = [
    Path('../data/raw/handwriting_personality_large_dataset.csv'),  # ejecución desde /notebooks
    Path('data/raw/handwriting_personality_large_dataset.csv')      # ejecución desde raíz del repo
]

DATA_PATH = next((p for p in path_candidates if p.exists()), path_candidates[0])

# Si ya existe un DataFrame llamado `df`, reutilizamos esa carga.
if 'df' in globals() and isinstance(df, pd.DataFrame):
    print('Se reutiliza el DataFrame `df` ya cargado en memoria.')
    df = df.copy()
else:
    df = pd.read_csv(DATA_PATH)
    print(f'Dataset cargado desde: {DATA_PATH}')

print(f'Filas: {df.shape[0]:,} | Columnas: {df.shape[1]:,}')


## 2) Exploración general

In [ ]:
display(df.head())

display(df.sample(5, random_state=42))

In [ ]:
print('Dimensiones del dataset (filas, columnas):', df.shape)
print()
print('Información general:')
df.info()


## 3) Tipos de variables

In [ ]:
dtypes_df = (
    df.dtypes
    .astype(str)
    .reset_index()
    .rename(columns={'index': 'variable', 0: 'dtype'})
)

display(dtypes_df)

print('Resumen por tipo de dato:')
print(dtypes_df['dtype'].value_counts())

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print(f'Variables numéricas ({len(numeric_cols)}): {numeric_cols}')
print(f'Variables categóricas/bool ({len(categorical_cols)}): {categorical_cols}')

## 4) Visualización descriptiva general

In [ ]:
# Estadísticos generales (incluyendo categóricas)
display(df.describe(include='all').T)

In [ ]:
# Distribución de tipos de variables
plt.figure(figsize=(8, 4))
(dtypes_df['dtype'].value_counts()).plot(kind='bar')
plt.title('Número de variables por tipo de dato')
plt.xlabel('Tipo de dato')
plt.ylabel('Número de variables')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5) Calidad básica: valores nulos

In [ ]:
nulls_abs = df.isna().sum()
nulls_pct = (nulls_abs / len(df) * 100).round(2)

nulls_report = (
    pd.DataFrame({'nulos': nulls_abs, 'pct_nulos': nulls_pct})
    .sort_values(['nulos', 'pct_nulos'], ascending=False)
)

display(nulls_report)

print('Variables con al menos 1 nulo:')
display(nulls_report[nulls_report['nulos'] > 0])

## 6) Valores únicos por variable

In [ ]:
unique_abs = df.nunique(dropna=False)
unique_pct = (unique_abs / len(df) * 100).round(2)

unique_report = (
    pd.DataFrame({'valores_unicos': unique_abs, 'pct_unicos_sobre_filas': unique_pct})
    .sort_values('valores_unicos', ascending=False)
)

display(unique_report)

## 7) Columnas constantes o poco informativas

In [ ]:
# Columnas constantes (1 valor único incluyendo NaN)
constant_cols = unique_abs[unique_abs <= 1].index.tolist()

# Heurística: columnas de muy baja variabilidad (<=1% de valores únicos respecto al total)
low_info_threshold = 1.0
low_info_cols = unique_report[
    unique_report['pct_unicos_sobre_filas'] <= low_info_threshold
].index.tolist()

print(f'Columnas constantes: {constant_cols if constant_cols else "Ninguna"}')
print(f'Columnas potencialmente poco informativas (<= {low_info_threshold}% valores únicos):')
print(low_info_cols if low_info_cols else 'Ninguna')

if low_info_cols:
    display(unique_report.loc[low_info_cols].sort_values('pct_unicos_sobre_filas'))

## 8) Conclusiones iniciales (Issue 1)

- Se validó la estructura general del dataset (tamaño y tipos de variables).
- Se revisó visualmente una muestra y estadísticas descriptivas básicas.
- Se generó un reporte de nulos por variable.
- Se generó un reporte de cardinalidad por variable.
- Se identificaron columnas constantes y columnas candidatas a ser poco informativas.

Con esto queda completo el **Issue 1** y el dataset listo para continuar con limpieza inicial en el siguiente apartado del EDA.
